In [8]:
import pandas as pd
import csv

path = '../crawl_data/khaosod.csv'

data = pd.read_csv(path, encoding="utf8") 
#data.shape
data.head()

,Unnamed: 0,post_page_id,post_id,post_time,post_link,post_message,post_like_count,post_share_count,post_reactor_count,post_comment_count,post_angry_count,post_love_count,post_sad_count,post_wow_count,post_haha_count,post_crawl_time,page_name
0,0,129558990394402,4333442143339378,2019-11-13 12:35:43,https://www.facebook.com/khaosod/posts/4333442...,เพื่อนบ้านรังเกียจ ลูกไม่ยอมให้เงิน เพราะกลัวจ...,1415,190,1777,205,2,10,326,19,5,2019-11-13 14:40:02.211674,khaosod
1,1,129558990394402,4333407490009510,2019-11-13 12:24:11,https://www.facebook.com/khaosod/videos/122637...,เดือดจัด!! ฟอร์จูนเนอร์ ชนแล้วหนี มอไซค์ตามไล่...,12608,3811,15338,2309,2060,6,96,364,204,2019-11-13 14:40:02.211674,khaosod
2,2,129558990394402,4333341343349458,2019-11-13 12:15:32,https://www.facebook.com/khaosod/posts/4333341...,กลุ่มผู้สูงวัย ร้องบิ๊กตู่ ขอขึ้นเบี้ยยังชีพ,1392,50,1864,504,8,4,5,40,415,2019-11-13 14:40:02.211674,khaosod
3,3,129558990394402,4333333783350214,2019-11-13 12:10:38,https://www.facebook.com/khaosod/posts/4333333...,ทุ่มทุนสร้าง เคว้ง ไทยซีรีส์เรื่องแรก บนเน็ตฟล...,304,3,310,4,0,2,0,3,1,2019-11-13 14:40:02.211674,khaosod
4,4,129558990394402,4333330193350573,2019-11-13 12:05:03,https://www.facebook.com/khaosod/posts/4333330...,ร้านเสริมสวยกระเจิง จงอางยาว 3 เมตร แผ่แม่เบี้...,349,7,385,11,0,0,0,28,8,2019-11-13 14:40:02.211674,khaosod


In [124]:
import csv

path = '../crawl_data/post interaction/khaosod_post_interact.csv'

data = pd.read_csv(path, encoding="utf8") 
#data.shape
#data.head()

#love = data.loc[data['type'] == 'Love']
#love.head()

#post_url = data.loc[data['post_url'] == 'https://www.facebook.com/khaosod/posts/4333442143339378']
#post_url.size
#post_url.head()

#user_url = data.loc[data['user_url'] == '/profile.php?id=100043671036759']
#user_url.head()

data.size

1644034

In [135]:
from tqdm import tqdm, tqdm_notebook

'''
    map[user_url][post_user_type] = user_url
'''
mapper = {}

print ("row: %d" % (data.size))

for row in tqdm_notebook(data.iterrows()):
    
    post_url = row[1]['post_url']
    type_t = row[1]['type']
    user_url = row[1]['user_url']

    if not post_url in mapper:
        mapper[post_url] = {}

    if not type_t in mapper[post_url]:
        mapper[post_url][type_t] = []
    
    mapper[post_url][type_t].append(user_url)
        
print ("Finished!")


0it [00:00, ?it/s]
1it [00:00,  7.67it/s]

row: 1644034



654it [00:00, 10.95it/s]
1189it [00:00, 15.63it/s]
1844it [00:00, 22.30it/s]
2521it [00:00, 31.81it/s]
3230it [00:00, 45.36it/s]
3784it [00:00, 64.57it/s]
4337it [00:00, 91.69it/s]
4861it [00:00, 129.98it/s]
5381it [00:01, 183.58it/s]
5891it [00:01, 258.10it/s]
6395it [00:01, 360.55it/s]
7071it [00:01, 503.55it/s]
7665it [00:01, 694.14it/s]
8230it [00:01, 916.17it/s]
8734it [00:01, 1209.30it/s]
9268it [00:01, 1574.74it/s]
9865it [00:01, 2021.03it/s]
10467it [00:02, 2523.83it/s]
11022it [00:02, 2947.11it/s]
11556it [00:02, 3389.09it/s]
12086it [00:02, 3775.47it/s]
12663it [00:02, 4211.85it/s]
13204it [00:02, 4481.54it/s]
13815it [00:02, 4869.36it/s]
14374it [00:02, 4410.26it/s]
14874it [00:02, 4165.96it/s]
15335it [00:03, 4066.25it/s]
15773it [00:03, 4045.20it/s]
16200it [00:03, 3783.45it/s]
16598it [00:03, 3598.38it/s]
16973it [00:03, 3549.87it/s]
17356it [00:03, 3628.40it/s]
17835it [00:03, 3913.04it/s]
18380it [00:03, 4273.96it/s]
18826it [00:03, 4317.21it/s]
19271it [00:04, 3996.96

Finished!


In [123]:
len(mapper)

183

In [159]:
# Create relation between users
user_map = {}

for post_url, post_data in tqdm(mapper.items()):
    print (post_url)
    for type_k, type_data in post_data.items():
        print (type_k)
        print (len(type_data))
        #print (type_data)
        for user_id_i in type_data:
            for user_id_j in type_data:
                if not user_id_i in user_map:
                    user_map[user_id_i] = {}
                if not user_id_j in user_map[user_id_i]:
                    user_map[user_id_i][user_id_j] = 0
                    
                user_map[user_id_i][user_id_j] += 1


  0%|          | 0/183 [00:00<?, ?it/s]

https://www.facebook.com/khaosod/posts/4333442143339378
Like
2018



  1%|          | 1/183 [00:02<07:02,  2.32s/it]

Sad
508
Wow
34
Love
17
Haha
6
Angry
4
https://www.facebook.com/khaosod/videos/1226371814224085/
Like
4000
Wow
771



  1%|          | 2/183 [00:11<12:58,  4.30s/it]

...

Angry
251
Wow
29
Sad
13
Love
4
https://www.facebook.com/khaosod/posts/4330401413643451
Like
620



 99%|█████████▉| 182/183 [05:33<00:01,  1.27s/it]

Haha
45
Wow
28
Sad
15
Angry
7
Love
1
https://www.facebook.com/khaosod/posts/4330387983644794
Like
390



100%|██████████| 183/183 [05:34<00:00,  1.06it/s]

Wow
7
Love
1
Haha
1


In [ ]:
from tqdm import tqdm, tqdm_notebook

# Get similarity set
out_data = []

for user_i, user_i_dat in tqdm_notebook(user_map.items()):
    i_action_count = len(user_map[user_i])
    #print ('i_action_count: %d' % (i_action_count))
    for user_j in user_map[user_i]:
        if user_i == user_j:
            continue
        j_action_count = len(user_map[user_j])
        #print ('j_action_count: %d' % (j_action_count))
        ij_action_count = user_map[user_i][user_j]
        #print ('ij_action_count: %d' % (ij_action_count))
        
        row = []
        row.append(user_i)
        row.append(user_j)
        row.append(ij_action_count)
        i_not_ij = i_action_count - ij_action_count
        row.append(i_not_ij)
        j_not_ij = j_action_count - ij_action_count
        row.append(j_not_ij)
        
        # Get jaccard similarity
        intersec = ij_action_count
        union = i_not_ij + ij_action_count + j_not_ij
        row.append(intersec / union)
        
        # Get overlapping coefficient
        min_ij = min((i_not_ij, j_not_ij))
        #print ('min_ij: %d' % min_ij)
        row.append(intersec / min_ij)
        
        out_data.append(row)
    
df = pd.DataFrame(out_data, columns = ["user_i", "user_j", "intersec", "i_not_intersec", "j_not_intersec", "jaccard_sim", "overlap_coef"])
df.head()

In [ ]:
#sort
df_by_overlap_coef = df.sort(['overlap_coef'], ascending=[0])
df_by_overlap_coef.head()